In [1]:
using LinearAlgebra, Plots, LaTeXStrings, Symbolics
 

In [2]:
E = 2.1E06
ν = 0.3

C = ((1 - ν) / ((1 + ν) * (1 - ν))) * E * [1 (ν/(1-ν)) 0; (ν/(1-ν)) 1 0; 0 0 ((1-2*ν)/(2*(1-ν)))]

3×3 Matrix{Float64}:
 1.61538e6  6.92308e5  0.0
 6.92308e5  1.61538e6  0.0
 0.0        0.0        4.61538e5

In [3]:
numNodosx = 3
numNodosy = 3
numNodosGlobales = numNodosx * numNodosy

longx = 3.0
longy = 3.0

numElementos = (numNodosx - 1) * (numNodosy - 1)
GDLGlobales = numNodosGlobales * 2

18

In [4]:

function matrizConectividadGlobal(numNodosx, numNodosy)
    numElementos = (numNodosx - 1) * (numNodosy - 1)
    # Matriz de conectividad para elementos de cuatro nodos bidimensionales 
    #(El numero de elementos podría variarse)
    matrizConectividad = zeros(Int, numElementos, 4)
    elemento = 1
    for j in 1:numNodosy-1
        for i in 1:numNodosx-1
            nodoInferiorIzquierdo::Int = i + ((j - 1) * numNodosx)
            matrizConectividad[elemento, :] = [(nodoInferiorIzquierdo + numNodosx + 1) (nodoInferiorIzquierdo + numNodosx) nodoInferiorIzquierdo (nodoInferiorIzquierdo + 1)]
            elemento += 1
        end
    end
    display(matrizConectividad)
    return matrizConectividad
end 


matrizConectividadGlobal (generic function with 1 method)

In [5]:
function mallador(numNodosx, numNodosy, longx, longy)

    numElementos = (numNodosx - 1) * (numNodosy - 1)
    numSegmentosx = numNodosx - 1
    numSegmentosy = numNodosy - 1
    dNodosx = longx / numSegmentosx
    dNodosy = longy / numSegmentosy
    numNodosGlobales = numNodosx * numNodosy

    matrizCoordGlobales = zeros(numNodosGlobales, 2)
    Coordy = 0
    nodo = 1
    for j in 1:numNodosy
        Coordx = 0
        for i in 1:numNodosx
            B = [Coordx Coordy]
            matrizCoordGlobales[nodo, :] = [Coordx Coordy]
            #matrizCoordGlobales[nodo, 2] = Coordy
            Coordx += dNodosx
            nodo += 1
        end
        Coordy += dNodosy
    end

    #=
    for j in 1:numNodosy
        Coordx = 0
        nodos = nodos + 1
        for i in 1:numNodosx
            B = [Coordx Coordy]
            matrizCoordGlobales = [matrizCoordGlobales; B]
            #matrizCoordGlobales[] = [matrizCoordGlobales; B]
            Coordx += dNodosx
        end
        Coordy += dNodosy
    end
    =#
    return matrizCoordGlobales
end

mallador (generic function with 1 method)

In [6]:
function matrizGdGL(numNodosx, numNodosy, mConectividadGlobal)
    numElementos = (numNodosx - 1) * (numNodosy - 1)
    numNodosGlobales = numNodosx * numNodosy
    mGdGL = zeros(Integer, 8, numElementos)
    for i in 1:numElementos
        j = 1
        while j <= 4
            zz = mConectividadGlobal[i, :]
            mGdGL[2*j-1, i] = zz[j] * 2 - 1
            mGdGL[2*j, i] = zz[j] * 2
            j += 1
        end
    end
    return mGdGL
end

matrizGdGL (generic function with 1 method)

In [7]:
mConectividadGlobal = matrizConectividadGlobal(numNodosx, numNodosy)
#display(mConectividadGlobal)
mGdLG = matrizGdGL(numNodosx, numNodosy, mConectividadGlobal)
display(mGdLG)
mCoordNodales = mallador(numNodosx, numNodosy, longx, longy)
#display(mCoordNodales)

4×4 Matrix{Int64}:
 5  4  1  2
 6  5  2  3
 8  7  4  5
 9  8  5  6

8×4 Matrix{Integer}:
  9  11  15  17
 10  12  16  18
  7   9  13  15
  8  10  14  16
  1   3   7   9
  2   4   8  10
  3   5   9  11
  4   6  10  12

9×2 Matrix{Float64}:
 0.0  0.0
 1.5  0.0
 3.0  0.0
 0.0  1.5
 1.5  1.5
 3.0  1.5
 0.0  3.0
 1.5  3.0
 3.0  3.0

In [8]:
begin
    @variables r s
    Dr = Differential(r)
    Ds = Differential(s)

    nFInt = 4
    h₁ = 1 / 4 * (1 + r) * (1 + s)
    h₂ = 1 / 4 * (1 - r) * (1 + s)
    h₃ = 1 / 4 * (1 - r) * (1 - s)
    h₄ = 1 / 4 * (1 + r) * (1 - s)

    H = [h₁ h₂ h₃ h₄]

    ∇H = [transpose([expand_derivatives(Dr(H[i])) for i in 1:nFInt]); transpose([expand_derivatives(Ds(H[i])) for i in 1:nFInt])]
end

2×4 Matrix{Num}:
 0.25(1 + s)  -0.25(1 + s)  -0.25(1 - s)   0.25(1 - s)
 0.25(1 + r)   0.25(1 - r)  -0.25(1 - r)  -0.25(1 + r)

In [9]:
nNodosInt = 9
GL9Q = [-sqrt(0.6) -sqrt(0.6); 0 -sqrt(0.6); sqrt(0.6) -sqrt(0.6); sqrt(0.6) 0; sqrt(0.6) sqrt(0.6);
    0 sqrt(0.6); -sqrt(0.6) sqrt(0.6); -sqrt(0.6) 0; 0 0]
#display(GL9Q)
wGL9Q = [5/9 5/9; 8/9 5/9; 5/9 5/9; 5/9 8/9; 5/9 5/9; 8/9 5/9; 5/9 5/9; 5/9 8/9; 8/9 8/9];

In [10]:
function matrizKGL(mConectividadGlobal, mCoordNodales, C, numElementos, GDLGlobales, mGdLG, GL9Q, wGL9Q)
KG = zeros(Float64, GDLGlobales, GDLGlobales) 
mNodalCoords = zeros(4, 2)

for el in 1:numElementos
    KE = zeros(Float64, 8, 8)
    GLE = mGdLG[:, el]
    numGLE = length(GLE)
    mConectividadElemental = mConectividadGlobal[el, :]
    for nodo in 1:4
        mNodalCoords[nodo, :] = mCoordNodales[mConectividadElemental[nodo], :]
    end

    x = expand((H*mNodalCoords[:, 1])[1])
    y = expand((H*mNodalCoords[:, 2])[1])

    dxdr = expand_derivatives(Dr(x))
    dxds = expand_derivatives(Ds(x))
    dydr = expand_derivatives(Dr(y))
    dyds = expand_derivatives(Ds(y))

    J = [dxdr dydr; dxds dyds] # Jacobiano
    J_inv = inv(J) # Inversa del Jacobiano
    J_det = det(J) # Determinante del Jacobiano

    B1 = transpose(∇H) * J_inv
    BL₀ = [B1[1, 1] 0 B1[2, 1] 0 B1[3, 1] 0 B1[4, 1] 0
        0 B1[1, 2] 0 B1[2, 2] 0 B1[3, 2] 0 B1[4, 2]
        B1[1, 2] B1[1, 1] B1[2, 2] B1[2, 1] B1[3, 2] B1[3, 1] B1[4, 2] B1[4, 1]]

    for i in 1:9
        Jg = substitute(J, Dict([r => GL9Q[i, 1], s => GL9Q[i, 2]]))
        Jg_inv = substitute(J_inv, Dict([r => GL9Q[i, 1], s => GL9Q[i, 2]]))
        Jg_det = substitute(J_det, Dict([r => GL9Q[i, 1], s => GL9Q[i, 2]]))
        Bg = substitute(BL₀, Dict([r => GL9Q[i, 1], s => GL9Q[i, 2]]))
        KE += (transpose(Bg) * C * Bg * Jg_det * wGL9Q[i, 1] * wGL9Q[i, 2])
    end

    for i in 1:numGLE
        for j in 1:numGLE
            KG[GLE[i], GLE[j]] += KE[i, j].val
        end
    end
end
return KG
end

matrizKGL (generic function with 1 method)

In [11]:
KG = matrizKGL(mConectividadGlobal, mCoordNodales, C, numElementos, GDLGlobales, mGdLG, GL9Q, wGL9Q)

18×18 Matrix{Float64}:
      6.92308e5       2.88462e5  …       0.0             0.0
      2.88462e5       6.92308e5          0.0             0.0
     -4.61538e5  -57692.3                0.0             0.0
  57692.3             1.15385e5          0.0             0.0
      0.0             0.0                0.0             0.0
      0.0             0.0        …       0.0             0.0
      1.15385e5   57692.3                0.0             0.0
 -57692.3            -4.61538e5          0.0             0.0
     -3.46154e5      -2.88462e5         -3.46154e5      -2.88462e5
     -2.88462e5      -3.46154e5         -2.88462e5      -3.46154e5
      0.0             0.0        …       1.15385e5   57692.3
      0.0             0.0           -57692.3            -4.61538e5
      0.0             0.0                0.0             0.0
      0.0             0.0                0.0             0.0
      0.0             0.0               -4.61538e5  -57692.3
      0.0             0.0        …   57692.3

In [12]:
#RE = zeros(Number, GDLGlobales, numElementos)
RG = zeros(GDLGlobales)
RG[18] = 3000
RG[14] = 3000
#RG[8] = 3000
#RG[6] = 3000
#RG = [0., 3000., 0., 3000., 0., 0., 0., 0.]
#for i in 1: 8 RG[mGdLG[i]] = RE[i] end
#display(typeof(RG))
#RG1 = convert.(AbstractFloat, RG)
#display(typeof(RG1))
display(RG)

18-element Vector{Float64}:
    0.0
    0.0
    0.0
    0.0
    0.0
    0.0
    0.0
    0.0
    0.0
    0.0
    0.0
    0.0
    0.0
 3000.0
    0.0
    0.0
    0.0
 3000.0

In [13]:
# Caso 1 (1 elemento)

# Imponemos condiciones de contorno de Dirichlet
#=
# U1x=0 => U1=0
KG[1, :] .= 0
KG[:, 1] .= 0
KG[1, 1] = 1

# U1y=0 => U2=0
KG[2, :] .= 0
KG[:, 2] .= 0
KG[2, 2] = 1

# U2x=0 => U3=0
KG[3, :] .= 0
KG[:, 3] .= 0
KG[3, 3] = 1

# U2y=0 => U4=0
KG[4, :] .= 0
KG[:, 4] .= 0
KG[4, 4] = 1

display(KG)

U₀ = KG \ RG # U₀ -> Desplazamientos en ⁰U 
=#

In [14]:
# Caso 2 (4 Elementos)
# Imponemos condiciones de contorno de Dirichlet

#
# U1x=0 => U1=0
KG[1, :] .= 0
KG[:, 1] .= 0
KG[1, 1] = 1

# U1y=0 => U2=0
KG[2, :] .= 0
KG[:, 2] .= 0
KG[2, 2] = 1

# U2x=0 => U3=0
KG[3, :] .= 0
KG[:, 3] .= 0
KG[3, 3] = 1

# U2y=0 => U4=0
KG[4, :] .= 0
KG[:, 4] .= 0
KG[4, 4] = 1

# U3x=0 => U5=0
KG[5, :] .= 0
KG[:, 5] .= 0
KG[5, 5] = 1

# U3y=0 => U6=0
KG[6, :] .= 0
KG[:, 6] .= 0
KG[6, 6] = 1

display(KG)

U = KG \ RG
#

18×18 Matrix{Float64}:
 1.0  0.0  0.0  0.0  0.0  0.0  …       0.0             0.0
 0.0  1.0  0.0  0.0  0.0  0.0          0.0             0.0
 0.0  0.0  1.0  0.0  0.0  0.0          0.0             0.0
 0.0  0.0  0.0  1.0  0.0  0.0          0.0             0.0
 0.0  0.0  0.0  0.0  1.0  0.0          0.0             0.0
 0.0  0.0  0.0  0.0  0.0  1.0  …       0.0             0.0
 0.0  0.0  0.0  0.0  0.0  0.0          0.0             0.0
 0.0  0.0  0.0  0.0  0.0  0.0          0.0             0.0
 0.0  0.0  0.0  0.0  0.0  0.0         -3.46154e5      -2.88462e5
 0.0  0.0  0.0  0.0  0.0  0.0         -2.88462e5      -3.46154e5
 0.0  0.0  0.0  0.0  0.0  0.0  …       1.15385e5   57692.3
 0.0  0.0  0.0  0.0  0.0  0.0     -57692.3            -4.61538e5
 0.0  0.0  0.0  0.0  0.0  0.0          0.0             0.0
 0.0  0.0  0.0  0.0  0.0  0.0          0.0             0.0
 0.0  0.0  0.0  0.0  0.0  0.0         -4.61538e5  -57692.3
 0.0  0.0  0.0  0.0  0.0  0.0  …   57692.3             1.15385e5
 0.0  0.0

18-element Vector{Float64}:
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.00038012514028317975
  0.0024876583744940484
 -3.8883276355802194e-19
  0.0013895381141987445
 -0.00038012514028318035
  0.002487658374494048
  0.0023806785862530177
  0.007497264615256223
  8.325420909912773e-20
  0.0012774191848092266
 -0.0023806785862530177
  0.007497264615256221